In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient("mongodb://172.17.0.2:27017/")

In [3]:
db = client.ecommerce
products = db.products

In [4]:
print(f"Database: {db.name}")
print(f"Collection: products")
print()

Database: ecommerce
Collection: products



In [5]:
# inserting initial products
products.insert_many([
    {"name": "Laptop", "price": 1200, "category": "Electronics", "inventory": 10},
    {"name": "T-Shirt", "price": 20, "category": "Clothing", "inventory": 50},
    {"name": "Sofa", "price": 700, "category": "Furniture", "inventory": 5, 
     "reviews": [{"user": "Alice", "rating": 4}]}
])

InsertManyResult([ObjectId('690c91a036d1b83eb957f3f5'), ObjectId('690c91a036d1b83eb957f3f6'), ObjectId('690c91a036d1b83eb957f3f7')], acknowledged=True)

In [6]:
# inserting 5 new porducts
new_products = [
    {"name": "Vacuum", "price": 800, "category": "Electronics", "inventory": 25, 
     "reviews": [{"user": "Bob", "rating": 5}]},
    {"name": "Speaker", "price": 150, "category": "Electronics", "inventory": 10},
    {"name": "Jacket", "price": 45, "category": "Clothing", "inventory": 40, 
     "discount": 10},
    {"name": "Table", "price": 200, "category": "Furniture", "inventory": 2},
    {"name": "Office Chair", "price": 380, "category": "Furniture", "inventory": 15}
]
result = products.insert_many(new_products)
print(f"Inserted {len(result.inserted_ids)} new products")
print()

Inserted 5 new products



In [9]:
# reading and filtering data
filtered_electronics = products.find({"category": "Electronics", "price": {"$gt": 500}})
for product in filtered_electronics:
    print(f"Product: {product['name']} - ${product['price']}")

Product: Laptop - $1200
Product: Vacuum - $800


In [10]:
# updating data
furniture_items = products.find({"category": "Furniture"})
for item in furniture_items:
    discount = 10
    discounted_price = item['price'] * (1 - discount/100)
    products.update_one(
        {"_id": item['_id']},
        {"$set": {"discount": discount, "price": discounted_price}}
    )

In [11]:
# deleting
result = products.delete_many({"inventory": {"$lt": 5}})
print(f"Deleted {result.deleted_count} products")

Deleted 1 products


In [12]:
products.update_one({"name": "T-Shirt"}, {"$set": {"reviews": [{"user": "Bob", "rating": 3}], "rating": 3.5}})
products.update_one({"name": "Speaker"},{"$set": {"rating": 4.5}})

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [13]:
# reading all data
all_products = products.find()
for product in all_products:
    print(f"\n{product['name']}:")
    print(f"  Category: {product['category']}")
    print(f"  Price: ${product['price']}")
    print(f"  Inventory: {product['inventory']}")
    if 'discount' in product:
        print(f"  Discount: {product['discount']}%")
    if 'reviews' in product:
        print(f"  Reviews: {len(product['reviews'])} review(s)")
    if 'rating' in product:
        print(f"  Rating: {product['rating']}")


Laptop:
  Category: Electronics
  Price: $1200
  Inventory: 10

T-Shirt:
  Category: Clothing
  Price: $20
  Inventory: 50
  Reviews: 1 review(s)
  Rating: 3.5

Sofa:
  Category: Furniture
  Price: $567.0
  Inventory: 5
  Discount: 10%
  Reviews: 1 review(s)

Vacuum:
  Category: Electronics
  Price: $800
  Inventory: 25
  Reviews: 1 review(s)

Speaker:
  Category: Electronics
  Price: $150
  Inventory: 10
  Rating: 4.5

Jacket:
  Category: Clothing
  Price: $45
  Inventory: 40
  Discount: 10%

Office Chair:
  Category: Furniture
  Price: $307.8
  Inventory: 15
  Discount: 10%


In [14]:
client.close()